In [10]:
import pandas as pd
import numpy as np

In [39]:
def read_reactome(file_name, gene_name_start = "ENSG0"):
    df = pd.read_csv(file_name, sep='\t', header=None)
    
    if gene_name_start == None:
        sub_df = df
    else:
        subset_vec = df[0].str.startswith(gene_name_start)
        sub_df = df.loc[subset_vec]
    
    genes_df = sub_df.groupby(1)[0].apply(list)
    names_df = sub_df.groupby(1)[3].max()
    
    out_df = pd.concat([genes_df,names_df], axis=1)
    out_df.columns = ['genes', 'pathway_name']
    
    return out_df


def ID_to_symbol(dataset):    
    symbols = pd.read_csv('../data/Gene_names.txt', index_col='Gene stable ID')
    symbols = symbols.drop_duplicates(keep=False)
    x = 0
    for gene in dataset:
        try:
            dataset[x] = symbols.loc[gene,'Gene name']
            x = x + 1
        except KeyError:
            dataset.loc[x,'Symbol'] = float('NaN')
            x = x + 1
    return dataset


In [61]:
reactome = read_reactome('../data/Ensembl2Reactome_All_Levels.txt', gene_name_start='ENSMUSG')
reactome = reactome.set_index('pathway_name')
pathways = ['CYP2E1 reactions','FMO oxidises nucleophiles', 'Tryptophan catabolism', 'PAOs oxidise polyamines to amines', 'Amino Acid conjugation','Conjugation of benzoate with glycine','Conjugation of carboxylic acids','Conjugation of salicylate with glycine']

TFs = pd.read_csv('../data/Transfactors/chip_atlas_analysis_list_CORRECTED.csv').loc[1255:1953,['Antigen']]
TFs.set_index('Antigen', inplace=True)

res = pd.DataFrame()
for pathway in pathways:
    genes = reactome.loc[pathway, 'genes']
    genessyms = ID_to_symbol(genes)
    genedf = pd.DataFrame(genessyms)
    genedf.set_index(0, inplace=True)
    overlap = genedf.index.intersection(TFs.index)
    res.loc[:,pathway] = overlap
res


,CYP2E1 reactions,FMO oxidises nucleophiles,Tryptophan catabolism,PAOs oxidise polyamines to amines,Amino Acid conjugation,Conjugation of benzoate with glycine,Conjugation of carboxylic acids,Conjugation of salicylate with glycine
